# Logistic Regression: aircraft_family - has_fatal_injury

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle


pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../../datasets/alaska_single_engine_clean.csv', low_memory=False)

In [3]:
ak.shape

(5590, 39)

In [4]:
predictor = 'aircraft_family'

In [5]:
target = 'has_fatal_injury'

In [6]:
ak = ak[[predictor, target]]

In [7]:
ak.columns

Index(['aircraft_family', 'has_fatal_injury'], dtype='object')

## Dummify Columns

In [8]:
def dummies(df, col_inference, category_to_drop):
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    columns_to_dummify_drop_first = [col for col in categorical_columns if col != col_inference]
    
    df_dummies = pd.get_dummies(df, columns=columns_to_dummify_drop_first, drop_first=True)
    
    df_dummies = pd.get_dummies(df_dummies, columns=[col_inference], drop_first=False)
    
    dummy_to_drop = f"{col_inference}_{category_to_drop}"
    if dummy_to_drop in df_dummies.columns:
        df_dummies.drop(columns=[dummy_to_drop], inplace=True)
    
    return df_dummies

In [9]:
ak_dummies_weather = dummies(ak, predictor, 'PIPER PA-18 SUPER CUB')

In [10]:
ak_dummies_weather.columns

Index(['has_fatal_injury', 'aircraft_family_AERONCA 15AC SEDAN',
       'aircraft_family_AERONCA 7AC CHAMPION',
       'aircraft_family_BEECHCRAFT 1900', 'aircraft_family_BELL 206',
       'aircraft_family_CESSNA 140', 'aircraft_family_CESSNA 150',
       'aircraft_family_CESSNA 152', 'aircraft_family_CESSNA 170',
       'aircraft_family_CESSNA 172 SKYHAWK',
       'aircraft_family_CESSNA 175 SKYLARK',
       'aircraft_family_CESSNA 177 CARDINAL',
       'aircraft_family_CESSNA 180 SKYWAGON',
       'aircraft_family_CESSNA 182 SKYLANE',
       'aircraft_family_CESSNA 185 SKYWAGON',
       'aircraft_family_CESSNA 206 STATIONAIR', 'aircraft_family_CESSNA 207',
       'aircraft_family_CESSNA 208 CARAVAN', 'aircraft_family_CESSNA 402',
       'aircraft_family_CHAMPION/ BELLANCA/ AMERICAN CHAMPION CITABRIA',
       'aircraft_family_CHAMPION/ BELLANCA/ AMERICAN CHAMPION SCOUT',
       'aircraft_family_DE HAVILLAND DHC-2 BEAVER',
       'aircraft_family_DE HAVILLAND DHC-3 OTTER',
       'airc

In [11]:
ak_dummies_weather.head()

,has_fatal_injury,aircraft_family_AERONCA 15AC SEDAN,aircraft_family_AERONCA 7AC CHAMPION,aircraft_family_BEECHCRAFT 1900,aircraft_family_BELL 206,aircraft_family_CESSNA 140,aircraft_family_CESSNA 150,aircraft_family_CESSNA 152,aircraft_family_CESSNA 170,aircraft_family_CESSNA 172 SKYHAWK,...,aircraft_family_PIPER PA-20 PACER,aircraft_family_PIPER PA-22 TRI-PACER,aircraft_family_PIPER PA-28 CHEROKEE,aircraft_family_PIPER PA-31 NAVAJO,aircraft_family_PIPER PA-32 CHEROKEE SIX,aircraft_family_ROBINSON HELICOPTER COMPANY R44 II,aircraft_family_STINSON 108,aircraft_family_TAYLORCRAFT B,aircraft_family_TAYLORCRAFT F-19 SPORTSMAN,aircraft_family_UNCOMMON FAMILY
0,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


## Train, Test, Split

In [12]:
X = ak_dummies_weather.drop(columns = target)
y = ak_dummies_weather[target]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)

## Baseline

In [14]:
1 - y.mean()

0.8967799642218247

## Benchmark Logistic Regression Model

In [15]:
pipe_log_reg_bench = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_bench', LogisticRegression(max_iter=1000))
])

In [16]:
pipe_log_reg_bench.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('log_reg_bench', LogisticRegression(max_iter=1000))])

In [17]:
pipe_log_reg_bench.score(X_train, y_train)

0.8962432915921288

In [18]:
pipe_log_reg_bench.score(X_test, y_test)

0.9016100178890877

## Tuned Logistic Regression Model

In [19]:
pipe_log_reg_tuned = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_tuned', LogisticRegression())
])

In [20]:
pipe_log_reg_tuned.get_params()

{'memory': None,
 'steps': [('sc', StandardScaler()), ('log_reg_tuned', LogisticRegression())],
 'verbose': False,
 'sc': StandardScaler(),
 'log_reg_tuned': LogisticRegression(),
 'sc__copy': True,
 'sc__with_mean': True,
 'sc__with_std': True,
 'log_reg_tuned__C': 1.0,
 'log_reg_tuned__class_weight': None,
 'log_reg_tuned__dual': False,
 'log_reg_tuned__fit_intercept': True,
 'log_reg_tuned__intercept_scaling': 1,
 'log_reg_tuned__l1_ratio': None,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__multi_class': 'auto',
 'log_reg_tuned__n_jobs': None,
 'log_reg_tuned__penalty': 'l2',
 'log_reg_tuned__random_state': None,
 'log_reg_tuned__solver': 'lbfgs',
 'log_reg_tuned__tol': 0.0001,
 'log_reg_tuned__verbose': 0,
 'log_reg_tuned__warm_start': False}

In [21]:
pipe_log_reg_tuned_params = {
    'log_reg_tuned__max_iter': [100, 1_000],
    'log_reg_tuned__C': np.linspace(0.001, 1, 5),
    'log_reg_tuned__penalty': ['l2']
}

In [22]:
gs= GridSearchCV(pipe_log_reg_tuned,
                                param_grid = pipe_log_reg_tuned_params,
                                cv = 5,
                               n_jobs=6)

In [23]:
%%time
gs.fit(X_train, y_train)

CPU times: user 180 ms, sys: 133 ms, total: 313 ms
Wall time: 2.04 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sc', StandardScaler()),
                                       ('log_reg_tuned',
                                        LogisticRegression())]),
             n_jobs=6,
             param_grid={'log_reg_tuned__C': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'log_reg_tuned__max_iter': [100, 1000],
                         'log_reg_tuned__penalty': ['l2']})

In [24]:
gs.best_params_

{'log_reg_tuned__C': 0.001,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__penalty': 'l2'}

In [25]:
gs.score(X_train, y_train)

0.8962432915921288

In [26]:
gs.score(X_test, y_test)

0.9016100178890877

In [27]:
y_pred = gs.predict(X_test)

In [28]:
confusion_matrix(y_test, y_pred)

array([[504,   0],
       [ 55,   0]])

In [29]:
print(classification_report(y_test, y_pred, digits = 5))

              precision    recall  f1-score   support

           0    0.90161   1.00000   0.94826       504
           1    0.00000   0.00000   0.00000        55

    accuracy                        0.90161       559
   macro avg    0.45081   0.50000   0.47413       559
weighted avg    0.81290   0.90161   0.85496       559



/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Coefficients

In [30]:
feature_names = list(X_train.columns)

In [31]:
coef_log_odds = gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0]

In [32]:
coef_odds = np.exp(gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0])

In [33]:
coef_log_odds_df = pd.DataFrame({'feature': feature_names, 'coefficient': coef_log_odds})

In [34]:
coef_odds_df = pd.DataFrame({'feature': feature_names, 'coefficient': coef_odds})

In [35]:
pd.set_option('display.max_rows', None)
coef_log_odds_df.head()

,feature,coefficient
0,aircraft_family_AERONCA 15AC SEDAN,-0.012080
1,aircraft_family_AERONCA 7AC CHAMPION,-0.018444
2,aircraft_family_BEECHCRAFT 1900,0.023712
3,aircraft_family_BELL 206,0.001995
4,aircraft_family_CESSNA 140,-0.028507


In [36]:
pd.set_option('display.max_rows', None)
coef_odds_df.sort_values(by = 'coefficient', ascending=False).head(100)

,feature,coefficient
48,aircraft_family_UNCOMMON FAMILY,1.068196
20,aircraft_family_DE HAVILLAND DHC-2 BEAVER,1.053443
41,aircraft_family_PIPER PA-28 CHEROKEE,1.047645
16,aircraft_family_CESSNA 208 CARAVAN,1.037527
17,aircraft_family_CESSNA 402,1.027879
43,aircraft_family_PIPER PA-32 CHEROKEE SIX,1.027523
2,aircraft_family_BEECHCRAFT 1900,1.023996
30,aircraft_family_MAULE M-5,1.022025
14,aircraft_family_CESSNA 206 STATIONAIR,1.014108
42,aircraft_family_PIPER PA-31 NAVAJO,1.013837


In [37]:
file_name = f"../../datasets/nolan_logreg_{predictor}-{target}_coef.csv"

coef_log_odds_df.to_csv(file_name, index=False)

## Pickle

In [38]:
file_name = f"../../pickles/nolan_logreg_{predictor}-{target}.pkl"

with open(file_name, 'wb') as f:
    pickle.dump(gs, f) 